In [ ]:
MLFLOW_TRACKING_URI = '../models/mlruns'
MLFLOW_RUN_ID = "a107782f5e814648af1d609b046aa929"

CLUSTERS_YAML_PATH = "../data/processed/features_skills_clusters_description.yaml"

In [ ]:
import sys
sys.path.append('../')

In [ ]:
import pandas as pd
import sys

sys.path.append('../')
from scripts.JobPrediction import *

# Simulate additional skills

### 1. Calculate base probs

In [ ]:
# current_skills = ['C', 'C++']
# current_skills = ['R', 'SQL', 'RStudio']
current_skills = ['Python', 'Pandas']

In [ ]:
# Initialize model
model = JobPrediction(MLFLOW_TRACKING_URI, MLFLOW_RUN_ID, CLUSTERS_YAML_PATH)

In [ ]:
# Calculate base probability
base_predictions = model.predict_jobs_probabilities(current_skills)
base_predictions.sort_values(ascending=False)

### 2. Calculate Probs for new skills

In [ ]:
# Get possible new skills
all_skills = pd.Series(model.get_all_skills())
new_skills = all_skills[~all_skills.isin(current_skills)].copy()

In [ ]:
new_skills

In [ ]:
simulated_results = []
for skill in new_skills:
    additional_skill_prob = model.predict_jobs_probabilities([skill] + current_skills)
    additional_skill_uplift = (additional_skill_prob - base_predictions) / base_predictions
    additional_skill_uplift.name = skill
    simulated_results.append(additional_skill_uplift)

In [ ]:
simulated_results = pd.DataFrame(simulated_results)

In [ ]:
simulated_results

### Recommend

In [ ]:
target_job = 'Data scientist or machine learning specialist'

In [ ]:
target_results = simulated_results[target_job].sort_values(ascending=False)
target_results

In [ ]:
threshold = 1
recommendations = target_results[target_results > threshold].index.tolist()

In [ ]:
print("Your current skills: " + str(current_skills))
print("Your target job: " + str(target_job))
print("You might also consider learning: " + str(recommendations))